In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x1c7b52e58b0>)

In [3]:
train_data = pd.read_table("./data/ratings_train.txt")
test_data = pd.read_table("./data/ratings_test.txt")

In [4]:
train_data.sample(5)

,id,document,label
113086,8828917,원작은 예술이었으나 이것은 그냥 킬링타임용 그이상도 그이하도 아님. 왜 굳이 리메이...,0
11552,5951295,유쾌하고 즐거우면서 아름다운 영화,1
12819,9895115,다양한 생각을 자연스럽게 인정하는 일본 그래서 문학이 발전하고 기발한 애니매이션도 ...,1
13871,10216097,오랜만에 다시 보았다....새롭기도하고 반갑기도했다,1
55540,9218022,너무재밋게보고있어요부모님이랑같이보는데 빵빵터지고 눈물샘도자극하네요..너무좋은프로그램...,1


In [5]:
train_data.drop_duplicates(subset=['document'],inplace=True)

In [6]:
train_data.isnull().values.any()

True

In [7]:
train_data.dropna(how = 'any')

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [8]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['document'] = train_data['document'].str.replace("^ +","")
train_data['document'].replace("",np.nan, inplace=True)
train_data = train_data.dropna(how='any')

C:\Users\sechang\AppData\Local\Temp/ipykernel_8904/2298155351.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\sechang\AppData\Local\Temp/ipykernel_8904/2298155351.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("^ +","")


In [9]:
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['document'] = test_data['document'].str.replace("^ +","")
test_data['document'].replace("",np.nan, inplace=True)
test_data = test_data.dropna(how='any')

C:\Users\sechang\AppData\Local\Temp/ipykernel_8904/2047020776.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\sechang\AppData\Local\Temp/ipykernel_8904/2047020776.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("^ +","")


In [10]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [11]:
X_train = []
okt = Okt()
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_train.append(stopwords_removed_sentence)

100%|██████████| 145393/145393 [14:31<00:00, 166.75it/s]


In [12]:
X_train[:3]

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다']]

In [13]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_test.append(stopwords_removed_sentence)

100%|██████████| 49575/49575 [02:33<00:00, 322.01it/s]


In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [15]:
tokenizer.word_index

{'영화': 1,
 '보다': 2,
 '을': 3,
 '없다': 4,
 '이다': 5,
 '있다': 6,
 '좋다': 7,
 '너무': 8,
 '다': 9,
 '정말': 10,
 '되다': 11,
 '재밌다': 12,
 '적': 13,
 '만': 14,
 '같다': 15,
 '진짜': 16,
 '로': 17,
 '아니다': 18,
 '않다': 19,
 '점': 20,
 '에서': 21,
 '만들다': 22,
 '나오다': 23,
 '연기': 24,
 '것': 25,
 '평점': 26,
 '내': 27,
 '최고': 28,
 '그': 29,
 '나': 30,
 '안': 31,
 '인': 32,
 '스토리': 33,
 '생각': 34,
 '못': 35,
 '왜': 36,
 '드라마': 37,
 '게': 38,
 '감동': 39,
 '사람': 40,
 '보고': 41,
 '이렇다': 42,
 '말': 43,
 '고': 44,
 '아깝다': 45,
 '더': 46,
 '배우': 47,
 '때': 48,
 'ㅋㅋ': 49,
 '아': 50,
 '감독': 51,
 '거': 52,
 '그냥': 53,
 '재미있다': 54,
 '요': 55,
 '재미': 56,
 '시간': 57,
 '내용': 58,
 '뭐': 59,
 '까지': 60,
 '중': 61,
 '주다': 62,
 '자다': 63,
 '하고': 64,
 '지루하다': 65,
 '재미없다': 66,
 '쓰레기': 67,
 '수': 68,
 '네': 69,
 '모르다': 70,
 '가다': 71,
 '들다': 72,
 '그렇다': 73,
 '싶다': 74,
 '지': 75,
 '작품': 76,
 '사랑': 77,
 '알다': 78,
 '하나': 79,
 '다시': 80,
 '마지막': 81,
 '볼': 82,
 '이건': 83,
 '정도': 84,
 '저': 85,
 '완전': 86,
 '오다': 87,
 'ㅋ': 88,
 'ㅠㅠ': 89,
 '많다': 90,
 '처음': 91,
 'ㅋㅋㅋ': 92,
 '장면': 9

In [16]:
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if value < threshold:
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [17]:
vocab_size = total_cnt - rare_cnt + 1
vocab_size

19416

In [19]:
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [20]:
X_train[:3]

[[50, 454, 16, 260, 659],
 [933, 457, 41, 602, 1, 214, 1449, 24, 961, 675, 19],
 [386, 2444, 2315, 5671, 2, 222, 9]]

In [21]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [22]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [23]:
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

145162
145162


C:\Users\sechang\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [24]:
def below_threshold_len(max_len, nested_list):
    count = 0 
    for sentence in nested_list:
        if len(sentence) <= max_len:
            count = count + 1
    print(f"전체 샘플 중 길이가 {max_len}이하인 샘플의 비율: {count/len(nested_list) * 100}")

In [25]:
below_threshold_len(30,X_train)

전체 샘플 중 길이가 30이하인 샘플의 비율: 94.31944999380003


In [26]:
X_train = pad_sequences(X_train, maxlen=30)
X_test = pad_sequences(X_test , maxlen=30)

In [27]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  50,
       454,  16, 260, 659])

In [28]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128 

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_bset_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
1815/1815 [==============================] - ETA: 0s - loss: 0.3906 - acc: 0.8217
Epoch 00001: saving model to best_model.h5
1815/1815 [==============================] - 318s 168ms/step - loss: 0.3906 - acc: 0.8217 - val_loss: 0.3533 - val_acc: 0.8448
Epoch 2/15
1815/1815 [==============================] - ETA: 0s - loss: 0.3284 - acc: 0.8572
Epoch 00002: saving model to best_model.h5
1815/1815 [==============================] - 262s 144ms/step - loss: 0.3284 - acc: 0.8572 - val_loss: 0.3329 - val_acc: 0.8572
Epoch 3/15
1815/1815 [==============================] - ETA: 0s - loss: 0.3028 - acc: 0.8719
Epoch 00003: saving model to best_model.h5
1815/1815 [==============================] - 218s 120ms/step - loss: 0.3028 - acc: 0.8719 - val_loss: 0.3347 - val_acc: 0.8535
Epoch 4/15
1815/1815 [==============================] - ETA: 0s - loss: 0.2835 - acc: 0.8821
Epoch 00004: saving model to best_model.h5
1815/1815 [==============================] - 156s 86ms/step - loss: 0.2835 

In [29]:
loaded_model = load_model('best_model.h5')
print(f"테스트 정확도: {loaded_model.evaluate(X_test, y_test)[1]}")

1550/1550 [==============================] - 54s 33ms/step - loss: 0.3720 - acc: 0.8449- ETA: 
테스트 정확도: 0.8448814749717712


In [32]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = 30)
    score = float(loaded_model.predict(pad_new))
    if score > 0.5:
        print(f"{score*100} 확률로 긍정 리뷰입니다.\n")
    else:
        print(f"{score*100} 확률로 부정 리뷰입니다.\n")

In [33]:
sentiment_predict("이 영화 개꿀잼 ㅋㅋㅋ")

96.25915288925171 확률로 긍정 리뷰입니다.



In [37]:
pre_labeling_df = pd.read_csv('./data/labeling.txt',sep='\t',names=['sentence','sentiment','objective'])

In [38]:
pre_labeling_df.sample(5)

,sentence,sentiment,objective
7631,매일 사용하지 않는데 켜져 있으면 건전지 닳을까 봐,1.0,1.0
2561,설령 저울위에 물건이 올려져 있더라도 분후에 꺼지게 되고 다시 사용하려면 전원버튼을...,1.0,1.0
7704,재료별로 도마를 각각 따로 사용하는게 좋다고해요,1.0,1.0
242,구매할 수 있는 링크는 아래에 첨부하겠습니다,1.0,1.0
1701,네 가능합니다,1.0,1.0


In [39]:
pre_labeling_sentences = pre_labeling_df['sentence'].values
pre_labeling_objectives = pre_labeling_df['objective'].values

In [42]:
for sentence in pre_labeling_sentences[:10]:
    print(sentence)
    sentiment_predict(sentence)

기존에 스타 전자저울 을 사용하고 있었는데 이게 어느날부터 상태가 안 좋더라구요
3.5246580839157104 확률로 부정 리뷰입니다.

건전지를 교체해보기도 하고 이래저래 만져봐도 도통 말을 듣지 않길래 정신건강을 위해 다시 저울을 구입했어요
0.754159688949585 확률로 부정 리뷰입니다.

원래는 스텐저울과 전자저울 두가지가 있었는데 스텐저울은 이미 한참 전에 망가졌어요
24.691736698150635 확률로 부정 리뷰입니다.

디자인보다는 튼튼한 것으로 당장 주문해도 내일 올 수 있는 것으로 단위로 측정할 수 있는 것이 필요했어요
95.19228339195251 확률로 긍정 리뷰입니다.

그래서 이마트몰 쓱배송으로 스타 전자 저울을 구입하게 되었어요
87.69639730453491 확률로 긍정 리뷰입니다.

기존에 사용하던 저울은 양푼을 올리면 넘게 무게를 먹고 들어가니깐 거의 단위로만 측정해야 했거든요
68.66858005523682 확률로 긍정 리뷰입니다.

별로 대단한게 아니더라도 만 측정 가능한 것과 까지 측정 가능한 것은 큰 차이가 있더라구요
3.553035855293274 확률로 부정 리뷰입니다.

올리다가 살짝 넘었는데 에러가 나면 저는 열불이 나더라구요
5.864810943603516 확률로 부정 리뷰입니다.

저울이라며이제 저울을 구입했으니 마음 편하게 조금 넘게도 올릴 수 있어서 행복해요
99.51575994491577 확률로 긍정 리뷰입니다.

제품 특징과 사용 방법은 박스 하단에 있더라구요
47.846755385398865 확률로 부정 리뷰입니다.



In [43]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = 30)
    score = float(loaded_model.predict(pad_new))
    if score > 0.5:
        return 2
    else:
        return 0

In [44]:
new_labeling_sentiments = [sentiment_predict(sentence) for sentence in pre_labeling_sentences]
new_labeling_sentiments = np.array(new_labeling_sentiments)

In [45]:
new_labeling_sentiments[:30]

array([0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0,
       2, 0, 2, 0, 0, 2, 0, 2])

In [47]:
obj_X_train = pre_labeling_sentences[:298]

In [49]:
obj_y_train = pre_labeling_objectives[:298]

In [51]:
obj_y_train[np.isnan(obj_y_train)] = 0

In [53]:
obj_y_train[obj_y_train == 2] = 1

In [57]:
obj_X_train = [re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", obj) for obj in obj_X_train]
obj_X_train = [okt.morphs(sentence) for sentence in obj_X_train]

In [60]:
obj_X_train = [[word for word in obj if not word in stopwords] for obj in obj_X_train]

In [86]:
new_tokenizer = Tokenizer()
new_tokenizer.fit_on_texts(pre_labeling_sentences)

In [66]:
new_encoded = new_tokenizer.texts_to_sequences(obj_X_train)

In [69]:
new_pads = pad_sequences(new_encoded, maxlen=30) 

In [87]:
new_vocab_size = len(new_tokenizer.word_index)

In [88]:
new_vocab_size

38186

In [89]:
new_model = Sequential()
new_model.add(Embedding(new_vocab_size, embedding_dim))
new_model.add(LSTM(hidden_units))
new_model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('new_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_bset_only=True)

new_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
new_history = new_model.fit(new_pads, obj_y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
4/4 [==============================] - ETA: 0s - loss: 0.6611 - acc: 0.6513
Epoch 00001: saving model to new_best_model.h5
4/4 [==============================] - 14s 1s/step - loss: 0.6611 - acc: 0.6513 - val_loss: 0.6449 - val_acc: 0.7167
Epoch 2/15
4/4 [==============================] - ETA: 0s - loss: 0.6069 - acc: 0.6975
Epoch 00002: saving model to new_best_model.h5
4/4 [==============================] - 1s 360ms/step - loss: 0.6069 - acc: 0.6975 - val_loss: 0.5934 - val_acc: 0.7167
Epoch 3/15
4/4 [==============================] - ETA: 0s - loss: 0.5833 - acc: 0.7101
Epoch 00003: saving model to new_best_model.h5
4/4 [==============================] - 1s 181ms/step - loss: 0.5833 - acc: 0.7101 - val_loss: 0.6276 - val_acc: 0.7167
Epoch 4/15
4/4 [==============================] - ETA: 0s - loss: 0.5095 - acc: 0.7689
Epoch 00004: saving model to new_best_model.h5
4/4 [==============================] - 1s 192ms/step - loss: 0.5095 - acc: 0.7689 - val_loss: 0.5856 - val_ac

In [90]:
new_loaded_model = load_model('new_best_model.h5')

In [91]:
def new_sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = 30)
    score = float(new_loaded_model.predict(pad_new))
    if score > 0.5:
        return 1
    else:
        return 0

In [92]:
new_labeling_objectives = [new_sentiment_predict(sentence) for sentence in pre_labeling_sentences]
new_labeling_objectives = np.array(new_labeling_objectives)

In [100]:
with open("./data/new_labeling.txt","w",encoding="utf-8") as f:
    for sentence, sentiment, objective in zip(pre_labeling_sentences, new_labeling_sentiments, new_labeling_objectives):
        f.write(f"{sentence}\t{sentiment}\t{objective}\n")

In [194]:
def training(X, y, output_dim, optimizer_function, loss_function, activation_function):
    model = Sequential()
    model.add(Embedding(50000, embedding_dim, input_length=X.shape[1]))
    model.add(LSTM(hidden_units))
    model.add(Dense(output_dim, activation=activation_function))

    model.compile(optimizer=optimizer_function, loss=loss_function, metrics=['acc'])
    history = model.fit(X, y, epochs=15, batch_size=64, validation_split=0.2)
    return model

In [146]:
def make_X_data(X,labeling_index):
    X = X[:labeling_index]
    X = [re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",sent) for sent in X]
    X = [okt.morphs(sent, stem=True) for sent in X]
    X = [sent for sent in X if not sent in stopwords]
    X = new_tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=50)
    return X

In [104]:
def make_y_data(y, labeling_index):
    y = y[:labeling_index]
    return y

In [105]:
def get_data_from_path(file_path):
    df = pd.read_csv(file_path, sep='\t' ,names=['sentence','sentiment','objective'])
    sentences = df['sentence'].values
    sentiments = df['sentiment'].values
    objectives = df['objective'].values
    return (sentences, sentiments, objectives)

In [176]:
sentences, sentiments, objectives = get_data_from_path("./data/new_labeling.txt")

In [177]:
training_sentences = make_X_data(sentences,504)

In [178]:
training_sentiments = make_y_data(sentiments,504)
training_objectives = make_y_data(objectives,504)

In [179]:
training_sentiments[np.isnan(training_sentiments)] = 1
training_sentiments

array([0., 0., 0., 2., 2., 2., 0., 0., 2., 0., 0., 2., 0., 2., 0., 2., 2.,
       0., 0., 2., 2., 0., 2., 0., 2., 0., 0., 2., 2., 0., 1., 2., 2., 0.,
       0., 2., 0., 0., 0., 2., 2., 2., 2., 0., 2., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 2., 2., 0., 2., 2., 0., 0., 2., 0., 0., 0., 0., 2., 2.,
       2., 0., 0., 2., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 1., 1., 1., 1., 2., 2., 2., 1., 2., 2., 2., 2., 2., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 2., 1., 1., 2., 2., 2., 1.,
       1., 2., 1., 1., 1., 2., 1., 0., 2., 1., 2., 1., 1., 2., 1., 1., 2.,
       2., 1., 2., 0., 2., 2., 1., 2., 2., 2., 1., 1., 2., 2., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 2., 2., 2., 2., 1.,
       1., 1., 1., 1., 1., 2., 2., 0., 2., 0., 0., 1., 2., 1., 2., 1., 0.,
       2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 1., 2., 1., 2., 1., 2., 2.,
       1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 2.,
       0., 2., 1., 2., 2.

In [210]:
training_objectives[np.isnan(training_objectives)] = 1
training_objectives

array([1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 0., 1., 0., 0.

In [182]:
def predict(X,model):
    return np.argmax(model.predict([sentence]))

In [139]:
def get_multi_y_train(y, num_of_classes):
    new_y = np.zeros((y.shape[0], num_of_classes))
    for idx, value in enumerate(y):
        value = int(value)
        new_y[idx][value] = 1
    return new_y

In [140]:
new_sentiments = get_multi_y_train(sentiments,3)

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [195]:
sentiment_model = training(training_sentences, new_sentiments, 3, 'adam', 'categorical_crossentropy', "softmax")

Epoch 1/15
7/7 [==============================] - 4s 324ms/step - loss: 1.0840 - acc: 0.4243 - val_loss: 0.9848 - val_acc: 0.5941
Epoch 2/15
7/7 [==============================] - 1s 177ms/step - loss: 1.0365 - acc: 0.4938 - val_loss: 0.9503 - val_acc: 0.4554
Epoch 3/15
7/7 [==============================] - 1s 182ms/step - loss: 1.0137 - acc: 0.4318 - val_loss: 0.9587 - val_acc: 0.4455
Epoch 4/15
7/7 [==============================] - 1s 184ms/step - loss: 0.9825 - acc: 0.4864 - val_loss: 0.9300 - val_acc: 0.5743
Epoch 5/15
7/7 [==============================] - 1s 204ms/step - loss: 0.9296 - acc: 0.6501 - val_loss: 0.9165 - val_acc: 0.6535
Epoch 6/15
7/7 [==============================] - 2s 215ms/step - loss: 0.8189 - acc: 0.6526 - val_loss: 1.2868 - val_acc: 0.6040
Epoch 7/15
7/7 [==============================] - 1s 174ms/step - loss: 0.6994 - acc: 0.7270 - val_loss: 1.1889 - val_acc: 0.6238
Epoch 8/15
7/7 [==============================] - 1s 165ms/step - loss: 0.5221 - acc: 0.81

In [216]:
predicted_sentences = make_X_data(sentences,sentences.shape[0]-1)

In [196]:
predicted_sentiments = sentiment_model.predict(predicted_sentences)

In [200]:
predicted_sentiments = np.array([np.argmax(sent) for sent in predicted_sentiments])

In [201]:
predicted_sentiments.shape

(7961,)

In [211]:
objective_model = training(training_sentences, training_objectives, 1, 'rmsprop', 'binary_crossentropy', "sigmoid")

Epoch 1/15
7/7 [==============================] - 8s 464ms/step - loss: 0.6626 - acc: 0.6948 - val_loss: 0.5034 - val_acc: 0.8119
Epoch 2/15
7/7 [==============================] - 1s 205ms/step - loss: 0.6240 - acc: 0.7072 - val_loss: 0.5441 - val_acc: 0.8119
Epoch 3/15
7/7 [==============================] - 2s 218ms/step - loss: 0.5576 - acc: 0.7072 - val_loss: 0.4721 - val_acc: 0.8119
Epoch 4/15
7/7 [==============================] - 2s 229ms/step - loss: 0.5214 - acc: 0.7097 - val_loss: 0.5277 - val_acc: 0.8218
Epoch 5/15
7/7 [==============================] - 1s 197ms/step - loss: 0.4673 - acc: 0.7519 - val_loss: 0.4707 - val_acc: 0.8119
Epoch 6/15
7/7 [==============================] - 1s 217ms/step - loss: 0.3851 - acc: 0.8139 - val_loss: 0.4828 - val_acc: 0.7723
Epoch 7/15
7/7 [==============================] - 1s 166ms/step - loss: 0.3300 - acc: 0.8859 - val_loss: 0.5393 - val_acc: 0.6931
Epoch 8/15
7/7 [==============================] - 1s 191ms/step - loss: 0.2921 - acc: 0.90

In [223]:
predicted_objectives = objective_model.predict(predicted_sentences)
predicted_objectives

array([[0.9994261 ],
       [0.999744  ],
       [0.99717903],
       ...,
       [0.9844115 ],
       [0.9841026 ],
       [0.9971274 ]], dtype=float32)

In [224]:
predicted_objectives = predicted_objectives.flatten()
predicted_objectives = np.array([1 if score>=0.5 else 0 for score in predicted_objectives])
predicted_objectives

array([1, 1, 1, ..., 1, 1, 1])

In [225]:
def save_labeling(file_path, data):
    sentences, sentiments, objectives = data
    with open(file_path,"w",encoding="utf-8") as f:
        for sentence, sentiment, objective in zip(sentences, sentiments, objectives):
            f.write(f"{sentence}\t{sentiment}\t{objective}\n")

In [226]:
save_labeling("./data/new_labeling2.txt",(sentences,predicted_sentiments,predicted_objectives))